In [1]:
!pip install keras


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install statsmodels

                                              0.0/9.2 MB ? eta -:--:--
                                              0.0/9.2 MB 660.6 kB/s eta 0:00:14
                                              0.0/9.2 MB 660.6 kB/s eta 0:00:14
                                              0.0/9.2 MB 279.3 kB/s eta 0:00:33
                                              0.1/9.2 MB 581.0 kB/s eta 0:00:16
                                              0.1/9.2 MB 554.9 kB/s eta 0:00:17
                                              0.2/9.2 MB 610.0 kB/s eta 0:00:15
                                              0.2/9.2 MB 655.1 kB/s eta 0:00:14
     -                                        0.2/9.2 MB 654.2 kB/s eta 0:00:14
     -                                        0.3/9.2 MB 706.2 kB/s eta 0:00:13
     -                                        0.3/9.2 MB 724.7 kB/s eta 0:00:13
     -                                        0.4/9.2 MB 716.8 kB/s eta 0:00:13
     -                                        0.4


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Prediction for future date

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import RandomForestRegressor
import warnings

# Load dataset
data = pd.read_csv('../output_deforestation_percentage_calculation.csv')  

# Convert date to datetime format and set it as index
data['date'] = pd.to_datetime(data['date'])  
data.set_index('date', inplace=True)

# Suppress specific warnings
warnings.filterwarnings("ignore", message="A date index has been provided")
warnings.filterwarnings("ignore", message="Maximum Likelihood optimization failed to converge")
warnings.filterwarnings("ignore", message="No supported index is available")

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data, test_data = data[:train_size], data[train_size:]

# Fit ARIMA model
order = (5, 1, 0)  #order (p, d, q)
model = ARIMA(train_data['Average_NDVI'], order=order)
model_fit = model.fit()

# Specify the specific date to predict for
specific_date = pd.to_datetime('2025-08-31')

# Forecast new NDVI value for the specific date
forecast_steps = 1  
forecast = model_fit.get_forecast(steps=forecast_steps)
forecasted_values = forecast.predicted_mean

# Convert the forecasted_values index to datetime
forecasted_values.index = pd.to_datetime(forecasted_values.index)

# Find the closest forecasted date to the specific_date
closest_date = (forecasted_values.index - specific_date).values.argmin()
closest_date = forecasted_values.index[closest_date]

# Get the forecasted NDVI value for the closest_date
new_ndvi = forecasted_values.loc[closest_date]

# Print forecasted NDVI value
print(f"Forecasted NDVI value for {specific_date}: {new_ndvi}")

# Train Random Forest model for deforestation percentage prediction
X = data[['Average_NDVI']]
y = data['Deforestation_Percentage']
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X, y)

# Predict deforestation percentage for the new NDVI value
new_data = pd.DataFrame({'Average_NDVI': [new_ndvi]})
new_deforestation_percentage = regressor.predict(new_data)[0]

print(f"Predicted deforestation percentage for {specific_date} = {new_deforestation_percentage} %")


Forecasted NDVI value for 2025-08-31 00:00:00: 0.29640140273373794
Predicted deforestation percentage for 2025-08-31 00:00:00 = 17.78553738248758 %


# Model Accuracy

In [2]:
# Use the trained regressor to make predictions on the test set
test_X = test_data[['Average_NDVI']]
test_y = test_data['Deforestation_Percentage']
predicted_deforestation_percentage = regressor.predict(test_X)

# Calculate the accuracy metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(test_y, predicted_deforestation_percentage)
mse = mean_squared_error(test_y, predicted_deforestation_percentage)
r2 = r2_score(test_y, predicted_deforestation_percentage)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2) Score: {r2}")

Mean Absolute Error (MAE): 0.29767006518816885
Mean Squared Error (MSE): 0.15406287452606163
R-squared (R2) Score: 0.9982258434435329
